<a href="https://colab.research.google.com/github/Alinehbg/CRP/blob/Anmol_Trial/Extract%20Names/Extract_Wiki_Data_names_try_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import requests
import pandas as pd
import os
import json

In [ ]:
dir = os.getcwd()

In [126]:
dir

'/Users/joshuafan/Documents/GitHub/CRP/Extract Names'

In [ ]:
# names = pd.read_excel('/content/drive/MyDrive/CRP/Datasets/Dataset.xlsx')
# names['Name'] = names.apply (lambda x: x['Name'].replace(".","") , axis = 1)
# names['Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")), axis = 1)
# names['First_Name_Length'] = names.apply(lambda x: len(x['Name'].split(" ")[0]), axis = 1)
# names = names.loc[names['Name_Length'] == 2]
# names = names.loc[names['First_Name_Length'] > 1]
# names['Name'] = names.apply(lambda x: x['Name'].title(), axis = 1 )
# names = names[["Name"]]
# names_list = list(names['Name'])
# names_list = list(set(names_list))
# names.to_csv('/content/drive/MyDrive/CRP/Datasets/Names_list.csv')

In [127]:
names_chunk = pd.read_csv('/Users/joshuafan/Documents/GitHub/CRP/Names_List.csv', chunksize = 10)

# functions 

In [6]:
import time
import datetime

In [137]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def retry(data): 
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  while response.status_code == 429:
    retry_after = response.headers.get("Retry-After")
    if retry_after:
      # If the value of "Retry-After" is a number, wait that many seconds
      if retry_after.isdigit():
        time.sleep(int(retry_after))
        # If the value of "Retry-After" is a date, parse it and wait until that time
      else:
        retry_time = datetime.datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
        if retry_time > datetime.datetime.now():
          time.sleep((retry_time - datetime.datetime.now()).total_seconds())
    # Try the request again
    response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  return res

def find_fname_id(name): 
  find_fn_id_query_male = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  find_fn_id_query_female = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query_male}
  res = retry(data)
  if res['results']['bindings'] == []: 
    data = {'query': find_fn_id_query_female}
    res = retry(data)
  try:
    id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
    #print(f"Cannot find first name id for {name}")
    id = []
  return id

def find_lname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  res = retry(data)
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find last name id for {name}")  
      id = []  
  return id

def find_similar_from_id(id): 
  #find similar names from the ID just found for the target name
  id = id
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  data = {'query': find_similar_query}
  res = retry(data)
  similar_name_list = get_itemlabel(res)
  return similar_name_list

exist_fname_path = '../Data files/first_names.txt'
exist_lname_path = '../Data files/last_names.txt'
similar_fname_path = '../Data files/first_names.json'
similar_lname_path = '../Data files/last_names.json'

def similar_full_names_separate(name): 
  fname_similar = []
  lname_similar = []
  f_name, l_name = name.split(' ')

  #open the existing first name and last name txt file and convert both to list
  with open(exist_fname_path, 'r') as f:
    first_names = f.readlines()
  exist_fname_list = []
  for line in first_names:
    exist_fname_list.append(line.strip())
  with open(exist_lname_path, 'r') as f:
    last_names = f.readlines()
  exist_lname_list = []
  for line in last_names:
    exist_lname_list.append(line.strip())
  
  #check if target first name and last name is in the existing list, if not, then proceed with search
  #if no id was found, then similar list will return empty else do search from id to find similar names
  if f_name not in exist_fname_list: 
    exist_fname_list.append(f_name)
    
    f_name_id = find_fname_id(f_name)
    if f_name_id == []: 
      fname_similar = []
    else: 
      fname_similar = find_similar_from_id(f_name_id)

    with open(exist_fname_path, 'w') as f:
      for line in exist_fname_list:
        f.write(line+'\n')
    
    with open(similar_fname_path, 'r+') as f:
            try:
                #load existing data from the file
                existing_data = json.load(f)
            except:
                #if file is empty set existing data to an empty dictionary
                existing_data = {}

            #add new key-value pair to the dictionary (will be in the same dict)
            existing_data[f_name] = fname_similar

            #write the updated dictionary back to the file
            f.seek(0) #move the file pointer to the beginning of the file (need to double check this)
            json.dump(existing_data, f, ensure_ascii=False)
            f.truncate() #truncate any remaining content in the file to avoid duplicates

  if l_name not in exist_lname_list: 
    exist_lname_list.append(l_name)
    l_name_id = find_lname_id(l_name)
    if l_name_id == []: 
      lname_similar = []
    else: 
      lname_similar = find_similar_from_id(l_name_id)

    with open(exist_lname_path, 'w') as f:
      for line in exist_lname_list:
        f.write(line+'\n')
    
    with open(similar_lname_path, 'r+') as f:
            try:
                #load existing data from the file
                existing_data = json.load(f)
            except:
                #if file is empty set existing data to an empty dictionary
                existing_data = {}

            #add new key-value pair to the dictionary (will be in the same dict)
            existing_data[l_name] = lname_similar

            #write the updated dictionary back to the file
            f.seek(0) #move the file pointer to the beginning of the file (need to double check this)
            json.dump(existing_data, f, ensure_ascii=False)
            f.truncate() #truncate any remaining content in the file to avoid duplicates

def Merge(dict1, dict2):
  return(dict1.update(dict2))

In [145]:
#create new files for existing name records and json file for saving similar names (4 files)
open(exist_fname_path, 'w') #if we already ran this first name before, it will be saved to this file
open(exist_lname_path, 'w')
open(similar_fname_path, 'w') #this is the file for similar first names. one dictionary with multiple keys (target first name)
open(similar_lname_path, 'w')

<_io.TextIOWrapper name='../Data files/last_names.json' mode='w' encoding='UTF-8'>

In [146]:
#clean all the txt and json files DO THIS WITH CARE

with open(exist_fname_path,'r+') as file:
    file.truncate(0)
with open(exist_lname_path,'r+') as file:
    file.truncate(0)
with open(similar_fname_path,'r+') as file:
    file.truncate(0)
with open(similar_lname_path,'r+') as file:
    file.truncate(0)

In [149]:
#check json and txt files to see the names are added 
#run joshua baker first then run joshua baker again. you will see nothing is added to the json and txt files
#then, run joshua baker and john watson together, you will see john watson is added to the json and txt files
similar_full_names_separate('Joshua Baker')
similar_full_names_separate('John Watson')

# requests and the rest

In [8]:
from tqdm import tqdm

In [9]:
FINAL_DF = pd.DataFrame(columns=['Target_Name', 'Alternate_Name', 'Match'])
i = 0

In [ ]:
for names in tqdm(names_chunk):
  name_list = names['Name'].tolist()
  name_dict = {}
  for name in name_list: 
    # temp = similar_full_names(name) (need to change this, function similar_full_names_separate now does not return but add to files directly)
    Merge(name_dict, temp)
  df = pd.DataFrame()

  Left = []
  Right = []
  Match = []

  Names = list(name_dict.keys())

  for name in Names:
    temp = Names
    non_names = temp.remove(name)

    for alternate in name_dict.get(name):
      Left.append(name)
      Right.append(alternate)
      Match.append(1)
    
    for non_name in temp:
      Left.append(name)
      Right.append(non_name)
      Match.append(0)
      for non_alternate in name_dict.get(non_name):
        Left.append(name)
        Right.append(non_alternate)
        Match.append(0)

  #To check
  assert len(Left) == len(Right)
  assert len(Left) == len(Match)

  df['Target_Name'] = Left
  df['Alternate_Name'] = Right
  df['Match'] = Match
  df.to_csv(dir + '/Batches/PART_' + str(i) + ".csv")
#   FINAL_DF = FINAL_DF.append(df)
  i += 1
#   if i == 10:
#     break



In [ ]:
FINAL_DF.info()

In [ ]:
FINAL_DF.to_csv("/content/drive/MyDrive/CRP/Datasets/FINAL_DF.csv")

In [ ]:
i

56